In [1]:
from sklearn.linear_model import Ridge

# 다항회귀
> 선형회귀의 경우 설명변수와 종속변수의 관계를 선형관계로 해석하지만 현실 문제는 선형으로 해결이 불가능한 경우가 많습니다.  
선형모델의 구조적 한계를 보완하고자 **변수 간 영향력에 해당하는 새로운 변수를 생성**하여 방정식을 비선형으로 만들고 선형모델을 적용시킬 수 있습니다.  
2차원 예시 $x_1, x_2, x_3$를 $x_1^2, x_2^2, x_3^2, x_1x_2, x_1x_3, x_2x_3$으로 변환하여 변수를 확장.  
보통 2차원 또는 3차원까지 적용하며, 변수가 많아지므로 Ridge, Lasso 모델을 적용한다.
<img src="./image/47.png">

### 다항회귀 실습

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
# boston 데이터 확인
df = pd.read_csv('./data/boston.csv')

In [4]:
# 타겟데이터 분할
y = df['y']
X = df.drop('y', axis=1)

In [7]:
# 훈련셋과 테스트셋 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
# 전처리 모델 import
from sklearn.preprocessing import PolynomialFeatures

In [9]:
# 다항변수 제작함수 인스턴스화
poly = PolynomialFeatures(degree=2, include_bias=False)
'''
파라메터
degree=2 : 차수설정
include_bias=False : 상수항 제거(필수) b0는 곱의 조합에서 뺄 것
'''

'\n파라메터\ndegree=2 : 차수설정\ninclude_bias=False : 상수항 제거(필수) b0는 곱의 조합에서 뺄 것\n'

In [ ]:
# 최대 표시 가능한 컬럼 갯수 설정 옵션
pd.options.display.max_columns= 200

In [16]:
# 변수 변환
# 전처리 모델이 학습이 필요한 부분은 설명변수의 갯수, 설명변수 이름
poly_X_train = pd.DataFrame(poly.fit_transform(X_train), columns=poly.get_feature_names_out()) # 데이터 학습 및 변환까지 한번에 진행하는 메소드

In [12]:
# 변환식 반환
poly.get_feature_names_out()

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT', 'CRIM^2', 'CRIM ZN', 'CRIM INDUS',
       'CRIM CHAS', 'CRIM NOX', 'CRIM RM', 'CRIM AGE', 'CRIM DIS',
       'CRIM RAD', 'CRIM TAX', 'CRIM PTRATIO', 'CRIM B', 'CRIM LSTAT',
       'ZN^2', 'ZN INDUS', 'ZN CHAS', 'ZN NOX', 'ZN RM', 'ZN AGE',
       'ZN DIS', 'ZN RAD', 'ZN TAX', 'ZN PTRATIO', 'ZN B', 'ZN LSTAT',
       'INDUS^2', 'INDUS CHAS', 'INDUS NOX', 'INDUS RM', 'INDUS AGE',
       'INDUS DIS', 'INDUS RAD', 'INDUS TAX', 'INDUS PTRATIO', 'INDUS B',
       'INDUS LSTAT', 'CHAS^2', 'CHAS NOX', 'CHAS RM', 'CHAS AGE',
       'CHAS DIS', 'CHAS RAD', 'CHAS TAX', 'CHAS PTRATIO', 'CHAS B',
       'CHAS LSTAT', 'NOX^2', 'NOX RM', 'NOX AGE', 'NOX DIS', 'NOX RAD',
       'NOX TAX', 'NOX PTRATIO', 'NOX B', 'NOX LSTAT', 'RM^2', 'RM AGE',
       'RM DIS', 'RM RAD', 'RM TAX', 'RM PTRATIO', 'RM B', 'RM LSTAT',
       'AGE^2', 'AGE DIS', 'AGE RAD', 'AGE TAX', 'AGE PTRATIO', 'AGE B',
       '

In [18]:
# test 데이터 동일한 모델로 적용
poly_X_test = poly.transform(X_test)

In [19]:
# 모델 정의
from sklearn.linear_model import Lasso

In [21]:
# 모델 학습
poly_lasso = Lasso()

In [22]:
# 모델 예측
# 학습은 다차항으로 변환한 데이터가 들어가야 함
poly_lasso.fit(poly_X_train, y_train)

C:\Users\studytsp86.edu\AppData\Local\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.006e+03, tolerance: 3.112e+00
  model = cd_fast.enet_coordinate_descent(


Lasso()

In [25]:
# 모델 평가
poly_lasso_pred = poly_lasso.predict(poly_X_test)

C:\Users\studytsp86.edu\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but Lasso was fitted with feature names
  warnings.warn(


In [26]:
from sklearn.metrics import r2_score, mean_squared_error
print(r2_score(y_test, poly_lasso_pred))
print(mean_squared_error(y_test, poly_lasso_pred, squared=False))

0.8254946931329348
3.605957654727187


In [ ]:
'''linear
0.7112260057484923
4.638689926172829
디폴트 릿지 모델
0.7041586727559439
4.695109486461522
패널티를 크게 준 모델
0.6954181695183629
4.763962116024234
'''

In [ ]:
# 성능은 뛰어남
# 통계모델링에 해당하는 기본 가정을 많이 깨가면서 모델링
# 예측성능에 해당하는 부분에 가장 초점을 맞추었기 때문에
# 프로젝트 목적이 예측이면 써볼만 함.
# 베타값들 해석, 비즈니스 액션을 취해야 하는 프로젝트 사용 X

In [ ]:
# 모델 계수 확인


In [ ]:
# 계수 플로팅
